In [5]:
import pandas as pd
import numpy as np

# Reading Excel files
file_path = 'North.xlsx'
data = pd.read_excel(file_path)

# Calculation of the number of districts with REPUBLICAN > DEMOCRAT
republican_greater_count = len(data[data['REPUBLICAN'] > data['DEMOCRAT']])

# Calculation of the number of districts with DEMOCRAT > REPUBLICAN
democrat_greater_count = len(data[data['DEMOCRAT'] > data['REPUBLICAN']])

# Calculate (REPUBLICAN - DEMOCRAT - other)/2) for all rows where (REPUBLICAN > DEMOCRAT) and store as array
republican_greater_df = data[data['REPUBLICAN'] > data['DEMOCRAT']]
result_array = ((republican_greater_df['REPUBLICAN'] - republican_greater_df['DEMOCRAT'] - republican_greater_df['other']) / 2).to_list()
result_array_ceil = np.ceil(result_array).astype(int).tolist()  # Round up and convert to integer

# Generate a dictionary, labelled with the district number and the value is the minimum number of changes
region_ids = republican_greater_df.index.tolist()
change_dict = {region_ids[i]: -result_array_ceil[i] for i in range(len(region_ids))}

# Calculate (DEMOCRAT - REPUBLICAN - other)/2) for all rows with DEMOCRAT > REPUBLICAN and store as an array of negative numbers.
democrat_greater_df = data[data['DEMOCRAT'] > data['REPUBLICAN']]
result_array_democrat = ((democrat_greater_df['DEMOCRAT'] - democrat_greater_df['REPUBLICAN'] - democrat_greater_df['other']) / 2).to_list()
result_array_ceil_democrat = np.ceil(result_array_democrat).astype(int).tolist()  # Round up and convert to negative

# Update the dictionary to add the region DEMOCRAT > REPUBLICAN
region_ids_democrat = democrat_greater_df.index.tolist()

# print(result_array_ceil_democrat)

for i in range(len(region_ids_democrat)):
    change_dict[region_ids_democrat[i]] = 1*result_array_ceil_democrat[i]

# output result
print("REPUBLICAN > DEMOCRAT :", republican_greater_count)
print("DEMOCRAT > REPUBLICAN :", democrat_greater_count)
print("REPUBLICAN > DEMOCRAT (REPUBLICAN - DEMOCRAT - other)/2 array:", result_array_ceil)

num_to_change = (republican_greater_count - democrat_greater_count) / 2 + 1



REPUBLICAN > DEMOCRAT : 8
DEMOCRAT > REPUBLICAN : 5
REPUBLICAN > DEMOCRAT (REPUBLICAN - DEMOCRAT - other)/2 array: [40514, 68801, 36128, 11866, 16552, 73956, 24534, 67167]


In [6]:
# Reading an Excel file and specifying column names
file_path = 'graph.xlsx'
df = pd.read_excel(file_path, header=None, names=['node', 'neighbors'])

# Get the number of nodes
num_nodes = df['node'].max()

# Initialise a two-dimensional array to store the adjacencies
adjacency_list = [[] for _ in range(num_nodes)]

# Parsing data and populating 2D arrays
for index, row in df.iterrows():
    node = row['node']
    neighbors = row['neighbors']
    # Split Neighbourhood Nodes
    neighbor_nodes = [(int(n)-1) for n in str(neighbors).split(',')]
    adjacency_list[node-1] = neighbor_nodes

# Convert to numpy array and print the result
adjacency_array = np.array(adjacency_list, dtype=object)
print(adjacency_array)


[list([2, 3]) list([6, 7]) list([0, 6]) list([0, 6, 7, 12])
 list([9, 10, 11]) list([9, 12]) list([0, 1, 2, 3, 7, 8])
 list([1, 3, 6, 8, 11, 12]) list([6, 7, 11]) list([4, 5, 11, 12])
 list([4]) list([4, 7, 8, 9]) list([3, 5, 7, 9])]


In [7]:
# Get the number of district with a positive value and sort them by absolute value from smallest to largest.
positive_regions = sorted((k for k, v in change_dict.items() if v > 0), key=lambda k: abs(change_dict[k]))

# Get the number of district with negative values and sort them in descending order of absolute value.
negative_regions = sorted((k for k, v in change_dict.items() if v < 0), key=lambda k: abs(change_dict[k]))

# Create a dictionary to store the results
result_dict = {}

# Iterate over positive district numbers
for supplier in positive_regions:
    supply_amount = change_dict[supplier]
    neighbors = adjacency_array[supplier]
    # Filtering out neighbouring negative area codes with an absolute value less than the current positive district code
    filtered_neighbors = [neighbor for neighbor in neighbors if neighbor in change_dict and change_dict[neighbor] < 0 and abs(change_dict[neighbor]) < supply_amount]
    result_dict[supplier] = filtered_neighbors

# Output Result Dictionary
print("Result dictionary:", result_dict)

Result dictionary: {0: [], 5: [], 1: [6, 7], 3: [6, 7, 12], 11: [4, 7, 8, 9]}


In [8]:

# Initialise num_to_change and total_change
num_to_change = (republican_greater_count - democrat_greater_count) // 2 + 1
total_change = 0

# Marks districts that have already been traversed
visited_regions = set()

# Traversing negative regions
for receiver in negative_regions:
    if num_to_change <= 0:
        break
    if receiver in visited_regions:
        continue
    
    # Find a positive area that can cover the negative region
    candidates = [supplier for supplier in result_dict if receiver in result_dict[supplier] and supplier not in visited_regions]
    if not candidates:
        continue
    
    # Select the region whose result_dict corresponds to the smallest length of the array, or the region with the smallest positive value if it is the same.
    selected_supplier = min(candidates, key=lambda supplier: (len(result_dict[supplier]), change_dict[supplier]))

    # Marks regions as traversed
    visited_regions.add(receiver)
    visited_regions.add(selected_supplier)
    
    # Update num_to_change and total_change
    num_to_change -= 1
    total_change += abs(change_dict[receiver])

if num_to_change > 0:
    print("unrealistic")
else:
    print("Total change needed:", total_change)

Total change needed: 28418
